In [ ]:
!pip install pandas dash
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/9b/b4/d522c16b41a8da013fd60a67f9618e57c504cd2c80e02a7a861413b93906/dash-2.13.0-py3-none-any.whl.metadata
  Using cached dash-2.13.0-py3-none-any.whl.metadata (11 kB)
  Using cached Flask-2.2.5-py3-none-any.whl (101 kB)
Using cached dash-2.13.0-py3-none-any.whl (10.4 MB)


--2023-09-05 01:12:28--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: 'spacex_launch_dash.csv.1'

     0K ..                                                    100%  176M=0s

2023-09-05 01:12:29 (176 MB/s) - 'spacex_launch_dash.csv.1' saved [2476/2476]

--2023-09-05 01:12:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-sto

In [ ]:


# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options = [{'label': 'All Sites', 'value': 'ALL'}, 
                                        {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'}, 
                                        {'label': 'KSC LC-39A', 'value':'KSC LC-39A'}, 
                                        {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'}],
                                    value=None,
                                    style={'textAlign': 'left', 'color': '#503D36' , 'width': '100%', 'padding': '3px', 'font-size': '15px'},
                                    placeholder='Select a Launch Site here',
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                        min=0, max=10000, step=1000,
                                        marks={0: '0',
                                            100: '100',
                                            1000: '1000',
                                            10000: '10000'},
                                        value=[0, 10000]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    df = spacex_df
    if entered_site in ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E']:    
        # return the outcomes piechart for a selected site
        df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        
        fig = px.pie(filtered_df, 
            values='class count', 
            names='class', 
            title=f"Success Launche Rate for Site: {entered_site}" )
        return fig 

    else:
        fig = px.pie(df, 
            values='class', 
            names='Launch Site', 
            title='Success Launch Rates for All Sites')
        return fig
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def get_scatter_chart(entered_site, entered_payload):
    #df = spacex_df
    low,high = (entered_payload[0], entered_payload[1])
    #df = spacex_df[spacex_df['Payload Mass (kg)']].between(low,high)
    #df = df.groupby(['Launch Site','class']).size().reset_index(name='class count')
    df=spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    

    if entered_site in ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E']:
        filtered_df = df[df['Launch Site'] == entered_site]
        
        fig = px.scatter(filtered_df, 
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category', 
            title=f'Success VS Payload Mass (kg) for: {entered_site}')
        return fig

    else: 
        fig = px.scatter(df, 
            x='Payload Mass (kg)', 
            y='class',
            color='Booster Version Category',
            hover_data=['Booster Version'], 
            title='Success VS Payload Mass (kg) for: All Sites')
        return fig
       

# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode="jupyterlab")
    app.run_server()
    
